In [1]:
import requests
from bs4 import BeautifulSoup
from robobrowser import RoboBrowser
import robobrowser
import csv 

#### Basic Scraping

Actual webpage for CSO events has the search functionality in an iframe of this page: http://apps.mwrd.org/csoreports/CSO_Synopisis_Report

Looks relatively straightforward, but using some kind of Microsoft Ajax library so a direct post doesn't seem to work. Looks like will have to GET the page first, get a few parameters (including a cookie) that all seem to be contained in hidden inputs and post from that

##### Example Arguments

* __EVENTTARGET:bttSearch
* __EVENTARGUMENT:
* __VIEWSTATE:"very long string"
* __VIEWSTATEGENERATOR:229E5B6F
* __EVENTVALIDATION: "long string"
* txtStartDateSearch:8/17/2016
* txtEndDateSearch:8/27/2016
* ReportViewer1$ctl03$ctl00:
* ReportViewer1$ctl03$ctl01:
* ReportViewer1$ctl10:ss:
* ReportViewer1$ctl11:standards
* ReportViewer1$AsyncWait$HiddenCancelField:False
* ReportViewer1$ToggleParam$store:
* ReportViewer1$ToggleParam$collapse:false
* ReportViewer1$ctl08$ClientClickedId:
* ReportViewer1$ctl07$store:
* ReportViewer1$ctl07$collapse:false
* ReportViewer1$ctl09$VisibilityState$ctl00:None
* ReportViewer1$ctl09$ScrollPosition:
* ReportViewer1$ctl09$ReportControl$ctl02:
* ReportViewer1$ctl09$ReportControl$ctl03:
* ReportViewer1$ctl09$ReportControl$ctl04:FullPage

In [2]:
mwrd_url = "http://apps.mwrd.org/csoreports/CSO_Synopisis_Report"
mwrd_page = requests.get(mwrd_url)
print(mwrd_page.content)

b'\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head><title>\r\n\r\n</title>\r\n    <style type="text/css">\r\n\r\n        .auto-style6 {\r\n            width: 832px;\r\n            margin-right: 4px;\r\n        }\r\n        .style33\r\n        {\r\n            font-family: "Segoe UI";\r\n            font-size: medium;\r\n            color: #0B73B5;\r\n            font-weight: bold;\r\n            border-left-color: #9D9DA1;\r\n            border-right-color: #C0C0C0;\r\n            border-top-color: #9D9DA1;\r\n            border-bottom-color: #C0C0C0;\r\n            padding: 1px;\r\n        }\r\n        .style34\r\n        {\r\n            font-family: "Segoe UI";\r\n            font-weight: bold;\r\n            font-size: medium;\r\n            color: #0099FF;\r\n        }\r\n        \r\n        .auto-style7 {\r\n            width: 172px;\r\n

In [3]:
mwrd_soup = BeautifulSoup(mwrd_page.content, 'html.parser')
input_vals = mwrd_soup.findAll('input')
for input_val in input_vals:
    print(input_val['name'])

__VIEWSTATE
__VIEWSTATEGENERATOR
__EVENTVALIDATION
bttSearchDay
bttResetSearch2
txtStartDateSearch
txtEndDateSearch
bttSearch
bttResetSearch
ReportViewer1$ctl03$ctl00
ReportViewer1$ctl03$ctl01
ReportViewer1$ctl10
ReportViewer1$ctl11
ReportViewer1$AsyncWait$HiddenCancelField
ReportViewer1$ToggleParam$img
ReportViewer1$ToggleParam$store
ReportViewer1$ToggleParam$collapse
ReportViewer1$ctl08$ClientClickedId
ReportViewer1$ctl07$img
ReportViewer1$ctl07$store
ReportViewer1$ctl07$collapse
ReportViewer1$ctl09$VisibilityState$ctl00
ReportViewer1$ctl09$ScrollPosition
ReportViewer1$ctl09$ReportControl$ctl02
ReportViewer1$ctl09$ReportControl$ctl03
ReportViewer1$ctl09$ReportControl$ctl04


In [4]:
s = requests.Session()
asp_headers = {
#     'Host': 'apps.mwrd.org',
#     'Origin': 'http://apps.mwrd.org',
#     'Referer': 'http://apps.mwrd.org/csoreports/CSO_Synopisis_Report',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
    'Connection': 'keep-alive',
    'Accept-Encoding': 'gzip, deflate'
}

mwrd_url = "http://apps.mwrd.org/csoreports/CSO_Synopisis_Report"
mwrd_page = s.get(mwrd_url)
mwrd_soup = BeautifulSoup(mwrd_page.content, 'html.parser')

all_inputs = mwrd_soup.find_all('input')

payload = {}

for inval in all_inputs:
    if inval.get('value'):
        payload[inval['name']] = inval['value']
    else:
        payload[inval['name']] = ''

payload['__EVENTTARGET'] = 'bttSearch'
payload['txtStartDateSearch'] = '08/13/2016'
payload['txtEndDateSearch'] = '08/27/2016'

payload['ReportViewer1$ctl11'] = 'standards'
#payload['ddlCSODates'] = '0'
#payload['ReportViewer1$ctl07$img'] = ''
# payload['__EVENTARGUMENT'] = ''
# payload['ReportViewer1$ctl09$VisibilityState$ctl00'] = 'ReportPage'

drop_keys = [
    'bttSearchDay',
    'bttResetSearch2',
    'bttSearch',
    'bttResetSearch',
    'ReportViewer1$ToggleParam$img'
]

for key in drop_keys:
    if payload.get(key):
        payload.pop(key)
# print('cookies: {} \n\n'.format(s.cookies))

for key in payload.keys():
    print(key)
# print(payload['ReportViewer1$ctl11'])
# print(payload['ReportViewer1$ctl09$VisibilityState$ctl00'])
mwrd_post_response = requests.post(mwrd_url, data=payload, headers=asp_headers, cookies=s.cookies)
mwrd_response_soup = BeautifulSoup(mwrd_post_response.content, 'html.parser')
# mwrd_response_soup = BeautifulSoup(resp.content, 'html.parser')
# print(mwrd_response_soup.find(id='ReportViewer1_ctl09'))

# Table with results has 9 cols, otherwise difficult to access
main_table_div = mwrd_response_soup.select("table[cols=9]")
# Get all table rows, other than the first which doesn't have the valign attribute, only searching in this subgroup
table_rows = main_table_div[0].select("tr[valign='top']")
# Create an empty list to add values to
table_list = []
# Go through each row, select the div in it, then grab the text from each div, add it to the row for the csv
for row in table_rows:
    row_cells = row.select('div')
    row_list = []
    # Get the text with the .text soup attribute, use .strip() to remove unnecessary whitespaces
    for cell in row_cells:
        row_list.append(cell.text.strip())
    table_list.append(row_list)

print(table_list)

txtStartDateSearch
ReportViewer1$ctl09$ScrollPosition
ReportViewer1$ctl03$ctl00
txtEndDateSearch
ReportViewer1$ToggleParam$collapse
ReportViewer1$ctl03$ctl01
ReportViewer1$ToggleParam$store
ReportViewer1$ctl09$ReportControl$ctl04
ReportViewer1$ctl11
ReportViewer1$ToggleParam$img
ReportViewer1$ctl09$ReportControl$ctl03
ReportViewer1$ctl07$img
ReportViewer1$AsyncWait$HiddenCancelField
ReportViewer1$ctl07$store
__VIEWSTATEGENERATOR
__EVENTVALIDATION
ReportViewer1$ctl10
ReportViewer1$ctl09$ReportControl$ctl02
__EVENTTARGET
ReportViewer1$ctl07$collapse
ReportViewer1$ctl09$VisibilityState$ctl00
__VIEWSTATE
ReportViewer1$ctl08$ClientClickedId
[['Outfall Structure', 'Outfall Location', 'Tide Gate #', 'Waterway Reach', 'Plant', 'Open date/time', 'Close date/time', 'Gate Open Period'], ['DS-D34-AI', 'Westchester P.S.(N)', 'All', 'Addison Cr', 'Pumping Station Westchester', '8/20/2016 8:40:00 AM', '8/20/2016 11:30:00 AM', ':02:50:00'], ['DS-D34-AI', 'Westchester P.S.(N)', 'All', 'Addison Cr', 'Pu

In [5]:
import csv 

with open('example_mwrd.csv', 'w') as mwrd_csv:
    mwrd_writer = csv.writer(mwrd_csv)
    for row in table_list:
        mwrd_writer.writerow(row)


In [6]:
# Create RoboBrowser, make sure to set parser to avoid errors later
browser = RoboBrowser(
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
    parser='html.parser'
)
# Open a session
browser.open('http://apps.mwrd.org/csoreports/CSO_Synopisis_Report')
# Identify the form by action, manually set important values
form = browser.get_form(action='./CSO_Synopisis_Report')
form['txtStartDateSearch'].value = '08/13/2016'
form['txtEndDateSearch'].value = '08/27/2016'
form['__EVENTTARGET'].value = 'bttSearch'
form['ReportViewer1$ctl11'].value = 'standards'
# form['ReportViewer1$ctl09$VisibilityState$ctl00'].value = 'ReportPage'

drop_keys = [
    'bttSearchDay',
    'bttResetSearch2',
    'bttSearch',
    'bttResetSearch',
    'ReportViewer1$ToggleParam$img'
]

# Remove extra keys
for key in drop_keys:
    form.fields.pop(key)

# Drop keys identified by printing all keys, then comparing to those posted in browser in Dev Tools
# for key in form.keys():
#     print('{}: {}'.format(key, form[key].value))

# Submit the form
browser.submit_form(form)

# Table with results has 9 cols, otherwise difficult to access
main_table_div = browser.select("table[cols=9]")
# Get all table rows, other than the first which doesn't have the valign attribute, only searching in this subgroup
table_rows = main_table_div[0].select("tr[valign='top']")
# Create an empty list to add values to
table_list = []
# Go through each row, select the div in it, then grab the text from each div, add it to the row for the csv
for row in table_rows:
    row_cells = row.select('div')
    row_list = []
    # Get the text with the .text soup attribute, use .strip() to remove unnecessary whitespaces
    for cell in row_cells:
        row_list.append(cell.text.strip())
    table_list.append(row_list)

print(table_list)

[['Outfall Structure', 'Outfall Location', 'Tide Gate #', 'Waterway Reach', 'Plant', 'Open date/time', 'Close date/time', 'Gate Open Period'], ['DS-D34-AI', 'Westchester P.S.(N)', 'All', 'Addison Cr', 'Pumping Station Westchester', '8/20/2016 8:40:00 AM', '8/20/2016 11:30:00 AM', ':02:50:00'], ['DS-D34-AI', 'Westchester P.S.(N)', 'All', 'Addison Cr', 'Pumping Station Westchester', '8/18/2016 7:00:00 AM', '8/18/2016 8:05:00 AM', ':01:05:00'], ['DS-M27,DS-M28,& DS-M29 (RAPS)', 'Racine Ave P.S. (S)', 'All', 'SF SB Chicago R', 'Pumping Station Racine Avenue', '8/20/2016 9:30:00 AM', '8/20/2016 8:25:00 PM', ':10:55:00'], ['DS-M90 & DS-M91 (NBPS)', 'North Branch P.S. (E)', 'All', 'NBCR Lower (NSC Confluence)', 'Pumping Station North Branch', '8/20/2016 9:14:00 AM', '8/20/2016 3:00:00 PM', ':05:46:00'], ['DS-M90 & DS-M91 (NBPS)', 'North Branch P.S. (E)', 'All', 'NBCR Lower (NSC Confluence)', 'Pumping Station North Branch', '8/13/2016 12:32:00 AM', '8/13/2016 2:31:00 AM', ':01:59:00']]


#### Solution

ASP.NET form is partially AJAX, but not entirely. Needed to ensure that the form values submitted by whatever library we were using are the same as what is submitted by the browser. A few values were different, and some fields that did not need values were being submitted.

Potentially most relevant, the key `ReportViewer1$ctl11` was blank on the first submit, but is `standards` in the successful post.

Helpful information on how this works was found in the Stack Overflow post here: http://stackoverflow.com/questions/27681731/python-requests-robobrowser-aspx-post-javascript

Many of the things that seemed mandatory were not, the most important factors were:

* Sending with a current session cookie
* Using `ReportViewer1$ctl11` 
* Setting the `__EVENTTARGET` to `bttSearch`
* Dropping unnecessary inputs that were associated